In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2
import urllib.parse
import datetime as dt
from urllib.parse import quote_plus as urlquote
from functools import reduce


In [2]:
covid_vaccine_csv = "../data/covid_vaccine.csv"
covid_cases_csv = "covid_cases.csv"

vaccine_df = pd.read_csv(covid_vaccine_csv)
cases_df = pd.read_csv(covid_cases_csv)

##covid_df = pd.merge(vaccine_df, cases_df)

In [3]:
cases_df


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-05,AK,305.0,NaN,2,NaN,1293.0,1293.0,33.0,3,...,1731628.0,7144,NaN,NaN,NaN,NaN,NaN,0,1731628.0,7144
1,2021-03-05,AL,10122.0,7943.0,28,2179.0,45976.0,45976.0,526.0,169,...,2316127.0,4920,NaN,NaN,119005.0,NaN,2316127.0,4920,NaN,0
2,2021-03-05,AR,5283.0,4291.0,10,992.0,14903.0,14903.0,359.0,27,...,2728103.0,8874,NaN,NaN,NaN,479119.0,NaN,0,2728103.0,8874
3,2021-03-05,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-05,AZ,16269.0,14339.0,84,1930.0,57819.0,57819.0,1043.0,72,...,7827570.0,51727,441664.0,NaN,NaN,NaN,3818450.0,20834,7827570.0,51727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20664,2020-01-16,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20665,2020-01-15,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20666,2020-01-14,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [4]:
vaccine_df

,date,state,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-03-03,Alabama,1000031.0,1469100.0,658906.0,6.91,20.40,338967.0,13.44,29.96,27870.0,22904.0,4671.0,0.681
1,2021-03-02,Alabama,972161.0,1461320.0,647752.0,6.57,19.83,322346.0,13.21,29.80,23768.0,22436.0,4576.0,0.665
2,2021-03-01,Alabama,948393.0,1402280.0,637989.0,6.29,19.34,308416.0,13.01,28.60,5891.0,22111.0,4510.0,0.676
3,2021-02-28,Alabama,942502.0,1402280.0,634792.0,6.24,19.22,305731.0,12.95,28.60,11412.0,21984.0,4484.0,0.672
4,2021-02-27,Alabama,931090.0,1402180.0,628583.0,6.13,18.99,300548.0,12.82,28.60,37231.0,22022.0,4491.0,0.664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3333,2021-01-16,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1328.0,1902.0,3286.0,NaN
3334,2021-01-15,Wyoming,22745.0,58775.0,19931.0,0.48,3.93,2803.0,3.44,10.15,2969.0,2093.0,3616.0,0.387
3335,2021-01-14,Wyoming,19776.0,58775.0,17310.0,NaN,3.42,NaN,2.99,10.15,1982.0,1654.0,2858.0,0.336
3336,2021-01-13,Wyoming,17794.0,57800.0,14406.0,0.44,3.08,2559.0,2.49,9.99,1327.0,1327.0,2293.0,0.308


In [5]:
cleaned_vaccine = vaccine_df.drop(columns=["total_distributed", "daily_vaccinations_raw", "share_doses_used"])
cleaned_vaccine

,date,state,total_vaccinations,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations,daily_vaccinations_per_million
0,2021-03-03,Alabama,1000031.0,658906.0,6.91,20.40,338967.0,13.44,29.96,22904.0,4671.0
1,2021-03-02,Alabama,972161.0,647752.0,6.57,19.83,322346.0,13.21,29.80,22436.0,4576.0
2,2021-03-01,Alabama,948393.0,637989.0,6.29,19.34,308416.0,13.01,28.60,22111.0,4510.0
3,2021-02-28,Alabama,942502.0,634792.0,6.24,19.22,305731.0,12.95,28.60,21984.0,4484.0
4,2021-02-27,Alabama,931090.0,628583.0,6.13,18.99,300548.0,12.82,28.60,22022.0,4491.0
...,...,...,...,...,...,...,...,...,...,...,...
3333,2021-01-16,Wyoming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1902.0,3286.0
3334,2021-01-15,Wyoming,22745.0,19931.0,0.48,3.93,2803.0,3.44,10.15,2093.0,3616.0
3335,2021-01-14,Wyoming,19776.0,17310.0,NaN,3.42,NaN,2.99,10.15,1654.0,2858.0
3336,2021-01-13,Wyoming,17794.0,14406.0,0.44,3.08,2559.0,2.49,9.99,1327.0,2293.0


In [6]:
abbv = {
"Alabama": "AL",
"Alaska": "AK",
"Arizona": "AZ",
"Arkansas": "AR",
"California": "CA",
"Colorado": "CO",
"Connecticut": "CT",
"Delaware": "DE",
"Florida": "FL",
"Georgia": "GA",
"Hawaii": "HI",
"Idaho": "ID",
"Illinois": "IL",
"Indiana": "IN",
"Iowa": "IA",
"Kansas": "KS",
"Kentucky":"KY",
"Louisiana": "LA",
"Maine" : "ME",
"Maryland": "MD",
"Massachusetts":"MA",
"Michigan" : "MI",
"Minnesota":"MN",
"Mississippi":"MS",
"Missouri":"MO",
"Montana":"MT",
"Nebraska":"NE",
"Nevada":"NV",
"New Hampshire":"NH",
"New Jersey":"NJ",
"New Mexico":"NM",
"New York":"NY",
"North Carolina":"NC",
"North Dakota":"ND",
"Ohio":"OH",
"Oklahoma":"OK",
"Oregon": "OR",
"Pennsylvania":"PA",
"Rhode Island":"RI",
"South Carolina":"SC",
"South Dakota":"SD",
"Tennessee":"TN",
"Texas":"TX",
"Utah":"UT",
"Vermont":"VT",
"Virginia":"VA",
"Washington":"WA",
"West Virginia": "WV",
"Wisconsin":"WI",
"Wyoming":"WY",
"American Samoa": "AS",
"Guam": "GU",
"Puerto Rico": "PR",
"Marshall Islands":"MH",
"Virgin Islands": "VI"
}
cleaned_vaccine['state'] = vaccine_df.state.replace(abbv)
cleaned_vaccine


,date,state,total_vaccinations,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations,daily_vaccinations_per_million
0,2021-03-03,AL,1000031.0,658906.0,6.91,20.40,338967.0,13.44,29.96,22904.0,4671.0
1,2021-03-02,AL,972161.0,647752.0,6.57,19.83,322346.0,13.21,29.80,22436.0,4576.0
2,2021-03-01,AL,948393.0,637989.0,6.29,19.34,308416.0,13.01,28.60,22111.0,4510.0
3,2021-02-28,AL,942502.0,634792.0,6.24,19.22,305731.0,12.95,28.60,21984.0,4484.0
4,2021-02-27,AL,931090.0,628583.0,6.13,18.99,300548.0,12.82,28.60,22022.0,4491.0
...,...,...,...,...,...,...,...,...,...,...,...
3333,2021-01-16,WY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1902.0,3286.0
3334,2021-01-15,WY,22745.0,19931.0,0.48,3.93,2803.0,3.44,10.15,2093.0,3616.0
3335,2021-01-14,WY,19776.0,17310.0,NaN,3.42,NaN,2.99,10.15,1654.0,2858.0
3336,2021-01-13,WY,17794.0,14406.0,0.44,3.08,2559.0,2.49,9.99,1327.0,2293.0


In [7]:
cases_df


,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-05,AK,305.0,NaN,2,NaN,1293.0,1293.0,33.0,3,...,1731628.0,7144,NaN,NaN,NaN,NaN,NaN,0,1731628.0,7144
1,2021-03-05,AL,10122.0,7943.0,28,2179.0,45976.0,45976.0,526.0,169,...,2316127.0,4920,NaN,NaN,119005.0,NaN,2316127.0,4920,NaN,0
2,2021-03-05,AR,5283.0,4291.0,10,992.0,14903.0,14903.0,359.0,27,...,2728103.0,8874,NaN,NaN,NaN,479119.0,NaN,0,2728103.0,8874
3,2021-03-05,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-05,AZ,16269.0,14339.0,84,1930.0,57819.0,57819.0,1043.0,72,...,7827570.0,51727,441664.0,NaN,NaN,NaN,3818450.0,20834,7827570.0,51727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20664,2020-01-16,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20665,2020-01-15,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20666,2020-01-14,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [8]:
start_date = "2021-1-12"
end_date = "2021-3-5"

after_start_date = cases_df["date"] >= start_date
before_end_date = cases_df["date"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = cases_df.loc[between_two_dates]
cases_df

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-05,AK,305.0,NaN,2,NaN,1293.0,1293.0,33.0,3,...,1731628.0,7144,NaN,NaN,NaN,NaN,NaN,0,1731628.0,7144
1,2021-03-05,AL,10122.0,7943.0,28,2179.0,45976.0,45976.0,526.0,169,...,2316127.0,4920,NaN,NaN,119005.0,NaN,2316127.0,4920,NaN,0
2,2021-03-05,AR,5283.0,4291.0,10,992.0,14903.0,14903.0,359.0,27,...,2728103.0,8874,NaN,NaN,NaN,479119.0,NaN,0,2728103.0,8874
3,2021-03-05,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-05,AZ,16269.0,14339.0,84,1930.0,57819.0,57819.0,1043.0,72,...,7827570.0,51727,441664.0,NaN,NaN,NaN,3818450.0,20834,7827570.0,51727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20664,2020-01-16,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20665,2020-01-15,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20666,2020-01-14,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [9]:
cleaned_cases = cases_df.drop(columns=["deathConfirmed","deathProbable","hospitalizedCumulative","hospitalizedCurrently","totalTestResultsIncrease","positiveTestsAntigen","positiveTestsPeopleAntibody","positiveTestsPeopleAntigen","positiveTestsViral","recovered","totalTestEncountersViral","totalTestEncountersViralIncrease","totalTestsViralIncrease","inIcuCumulative","inIcuCurrently", "negative","negativeIncrease","negativeTestsAntibody","negativeTestsPeopleAntibody","negativeTestsViral",
                      "onVentilatorCumulative","onVentilatorCurrently", "positiveCasesViral","positiveScore", "positiveTestsAntibody",
                      "totalTestsAntibody","totalTestsAntigen","totalTestsPeopleAntibody","totalTestsPeopleAntigen","totalTestsPeopleViral","totalTestsPeopleViralIncrease","totalTestsViral"])
cleaned_cases

,date,state,death,deathIncrease,hospitalized,hospitalizedIncrease,positive,positiveIncrease,totalTestResults
0,2021-03-05,AK,305.0,2,1293.0,3,56886.0,141,1731628.0
1,2021-03-05,AL,10122.0,28,45976.0,169,498887.0,811,2316127.0
2,2021-03-05,AR,5283.0,10,14903.0,27,324326.0,570,2728103.0
3,2021-03-05,AS,0.0,0,NaN,0,0.0,0,2140.0
4,2021-03-05,AZ,16269.0,84,57819.0,72,823384.0,2276,7827570.0
...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,0,NaN,0,0.0,0,NaN
20664,2020-01-16,WA,NaN,0,NaN,0,0.0,0,NaN
20665,2020-01-15,WA,NaN,0,NaN,0,0.0,0,NaN
20666,2020-01-14,WA,NaN,0,NaN,0,0.0,0,NaN


In [10]:
cleaned_cases_cols = ["date","state","death","deathIncrease","hospitalized","hospitalizedIncrease","positive","positiveIncrease","totalTestResults"]
cleaned_cases_transformed= cleaned_cases[cleaned_cases_cols].copy()
cleaned_cases_transformed = cleaned_cases_transformed.rename(columns={
                                                     "date": "date",
                                                     "state": "state",
                                                     "death": "death",
                                                     "deathIncrease": "death_Increase",
                                                     "hospitalized": "hospitalized",
                                                     "hospitalizedIncrease": "hospitalized_Increase",
                                                     "positive": "positive",
                                                     "positiveIncrease": "positive_Increase",})
                    
                    
cleaned_cases_transformed                  

,date,state,death,death_Increase,hospitalized,hospitalized_Increase,positive,positive_Increase,totalTestResults
0,2021-03-05,AK,305.0,2,1293.0,3,56886.0,141,1731628.0
1,2021-03-05,AL,10122.0,28,45976.0,169,498887.0,811,2316127.0
2,2021-03-05,AR,5283.0,10,14903.0,27,324326.0,570,2728103.0
3,2021-03-05,AS,0.0,0,NaN,0,0.0,0,2140.0
4,2021-03-05,AZ,16269.0,84,57819.0,72,823384.0,2276,7827570.0
...,...,...,...,...,...,...,...,...,...
20663,2020-01-17,WA,NaN,0,NaN,0,0.0,0,NaN
20664,2020-01-16,WA,NaN,0,NaN,0,0.0,0,NaN
20665,2020-01-15,WA,NaN,0,NaN,0,0.0,0,NaN
20666,2020-01-14,WA,NaN,0,NaN,0,0.0,0,NaN


In [11]:
cleaned_cases_transformed
cleaned_vaccine

cleaned_cases_transformed['date'] = pd.to_datetime(cleaned_cases_transformed['date'])
cleaned_vaccine['date'] = pd.to_datetime(cleaned_vaccine['date'])
merged_df = pd.merge(cleaned_cases_transformed, cleaned_vaccine, on= ["date" , "state"])
pd
merged_df

,date,state,death,death_Increase,hospitalized,hospitalized_Increase,positive,positive_Increase,totalTestResults,total_vaccinations,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations,daily_vaccinations_per_million
0,2021-03-03,AK,303.0,1,1285.0,-1,56605.0,177,1711018.0,273350.0,170243.0,14.01,37.37,102472.0,23.27,54.57,4243.0,5800.0
1,2021-03-03,AL,10029.0,38,45723.0,24,497154.0,2733,2305317.0,1000031.0,658906.0,6.91,20.40,338967.0,13.44,29.96,22904.0,4671.0
2,2021-03-03,AR,5261.0,7,14826.0,31,323353.0,404,2709946.0,687433.0,441126.0,8.03,22.78,242346.0,14.62,31.97,18995.0,6294.0
3,2021-03-03,AS,0.0,0,NaN,0,0.0,0,2140.0,21079.0,12949.0,14.54,37.85,8099.0,23.25,52.88,191.0,3430.0
4,2021-03-03,AZ,16089.0,29,57697.0,106,819954.0,1284,7739361.0,1930999.0,1308334.0,8.43,26.53,613681.0,17.97,32.87,53580.0,7361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,2021-01-12,VT,158.0,2,NaN,0,9247.0,169,767325.0,25757.0,NaN,NaN,4.13,NaN,NaN,10.40,NaN,NaN
2699,2021-01-12,WA,3699.0,1,15978.0,207,276686.0,5091,4087531.0,195567.0,162105.0,0.23,2.57,17689.0,2.13,7.46,NaN,NaN
2700,2021-01-12,WI,5633.0,58,22583.0,149,555249.0,3307,5644935.0,137253.0,125895.0,0.19,2.36,11343.0,2.16,7.38,NaN,NaN
2701,2021-01-12,WV,1634.0,40,NaN,0,103203.0,921,1661864.0,103330.0,NaN,NaN,5.77,NaN,NaN,8.98,NaN,NaN


In [12]:
connection_string = "postgres:Cellxan69$$@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{connection_string}')

In [13]:
engine.table_names()


['covid']

In [16]:
merged_df.to_sql(name= 'covid', con=engine, if_exists= 'replace', index= False)

In [ ]:
#cleaned_cases.to_sql(name= "cases", con=engine, if_exists= 'append', index= True)